In [1]:
#importing useful packages
import pandas as pd
from google.colab import files
import io
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark-palette')
import datetime
import sklearn
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
import math
import re

In [2]:
#USE THIS CELL TO PULL DATA FROM GOOGLE DRIVE
#mount google colab
from google.colab import drive
drive.mount('/content/gdrive')

#import data from google drive
clean = pd.read_csv('/content/gdrive/My Drive/urban economics/clean_station_hrly.csv')
print(clean.columns)
print(clean.shape)
clean.head()

Mounted at /content/gdrive
Index(['Timestamp', 'StationID', 'FreewayNum', 'DirectionOfTravel', 'LaneType',
       'StationLength', 'Samples', '%Observed', 'TotalFlow', 'AvgOccupancy',
       'AvgSpeed', 'numLanes', 'flowPerLane', 'DateTime', 'Hour', 'Day'],
      dtype='object')
(442611, 16)


,Timestamp,StationID,FreewayNum,DirectionOfTravel,LaneType,StationLength,Samples,%Observed,TotalFlow,AvgOccupancy,AvgSpeed,numLanes,flowPerLane,DateTime,Hour,Day
0,10/01/2020 00:00:00,308512,50,W,ML,3.995,225,0,447.0,0.0245,63.6,2.0,223.500000,2020-10-01 00:00:00,0,1
1,10/01/2020 00:00:00,311903,50,E,ML,0.883,356,100,905.0,0.0197,64.9,3.0,301.666667,2020-10-01 00:00:00,0,1
2,10/01/2020 00:00:00,311974,50,E,ML,0.466,595,100,1105.0,0.0179,64.4,5.0,221.000000,2020-10-01 00:00:00,0,1
3,10/01/2020 00:00:00,312010,50,W,ML,1.218,595,100,927.0,0.0141,67.4,5.0,185.400000,2020-10-01 00:00:00,0,1
4,10/01/2020 00:00:00,312098,50,E,ML,0.483,476,100,542.0,0.0082,68.7,4.0,135.500000,2020-10-01 00:00:00,0,1


In [3]:

#import data from google drive
meta = pd.read_table('/content/gdrive/My Drive/urban economics/metadata_2020_19_oct.txt')
meta.rename(columns={"ID": "StationID"}, inplace=True)
print(meta.columns)
print(meta.shape)
meta.head()

Index(['StationID', 'Fwy', 'Dir', 'District', 'County', 'City', 'State_PM',
       'Abs_PM', 'Latitude', 'Longitude', 'Length', 'Type', 'Lanes', 'Name',
       'User_ID_1', 'User_ID_2', 'User_ID_3', 'User_ID_4'],
      dtype='object')
(1291, 18)


,StationID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
0,308511,50,E,3,17,NaN,31.627,60.162,38.761062,-120.569835,5.000,ML,2,Sly Park Rd,1,NaN,NaN,NaN
1,308512,50,W,3,17,NaN,31.627,60.166,38.761182,-120.569866,3.995,ML,2,Sly Park Rd,1,NaN,NaN,NaN
2,311831,5,S,3,67,NaN,10.896,506.189,38.409782,-121.484120,NaN,OR,1,Elk Grove Blvd,1,NaN,NaN,NaN
3,311832,5,S,3,67,NaN,10.896,506.189,38.409782,-121.484120,NaN,FR,1,Elk Grove Blvd,1,NaN,NaN,NaN
4,311844,5,N,3,67,NaN,11.105,506.398,38.412779,-121.484363,NaN,OR,3,Elk Grove Blvd 5NB Slip,1,NaN,NaN,NaN


In [4]:
merged = clean.merge(right=meta, how='inner', on='StationID')
print(merged.shape)
merged.head()

(441071, 33)


,Timestamp,StationID,FreewayNum,DirectionOfTravel,LaneType,StationLength,Samples,%Observed,TotalFlow,AvgOccupancy,AvgSpeed,numLanes,flowPerLane,DateTime,Hour,Day,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,Length,Type,Lanes,Name,User_ID_1,User_ID_2,User_ID_3,User_ID_4
0,10/01/2020 00:00:00,308512,50,W,ML,3.995,225,0,447.0,0.0245,63.6,2.0,223.5,2020-10-01 00:00:00,0,1,50,W,3,17,NaN,31.627,60.166,38.761182,-120.569866,3.995,ML,2,Sly Park Rd,1,NaN,NaN,NaN
1,10/01/2020 01:00:00,308512,50,W,ML,3.995,227,0,430.0,0.0240,63.6,2.0,215.0,2020-10-01 01:00:00,1,1,50,W,3,17,NaN,31.627,60.166,38.761182,-120.569866,3.995,ML,2,Sly Park Rd,1,NaN,NaN,NaN
2,10/01/2020 02:00:00,308512,50,W,ML,3.995,233,0,435.0,0.0241,63.7,2.0,217.5,2020-10-01 02:00:00,2,1,50,W,3,17,NaN,31.627,60.166,38.761182,-120.569866,3.995,ML,2,Sly Park Rd,1,NaN,NaN,NaN
3,10/01/2020 03:00:00,308512,50,W,ML,3.995,229,0,451.0,0.0246,63.6,2.0,225.5,2020-10-01 03:00:00,3,1,50,W,3,17,NaN,31.627,60.166,38.761182,-120.569866,3.995,ML,2,Sly Park Rd,1,NaN,NaN,NaN
4,10/01/2020 04:00:00,308512,50,W,ML,3.995,215,0,528.0,0.0268,63.8,2.0,264.0,2020-10-01 04:00:00,4,1,50,W,3,17,NaN,31.627,60.166,38.761182,-120.569866,3.995,ML,2,Sly Park Rd,1,NaN,NaN,NaN


In [5]:
merged.to_csv('/content/gdrive/My Drive/urban economics/clean_meta_merge.csv', index=False)

In [6]:
#import data from google drive
closures = pd.read_excel('/content/gdrive/My Drive/urban economics/closures.xlsx')

closures = closures.dropna(axis=0, subset=['Begin Abs PM', 'End Abs PM'])

closures['StartDateTime'] =  pd.to_datetime(closures['Start Date'])
closures['EndDateTime'] =  pd.to_datetime(closures['End Date'])

print(closures.columns)
print(closures.shape)
#print(closures['StartDateTime'][0].type)
closures.head(20)

Index(['ID', 'Log #', 'District', 'Fwy-Dir', 'Begin County', 'Begin Abs PM',
       'Begin State PM', 'End County', 'End Abs PM', 'End State PM', 'Length',
       'Work Type', 'Start Date', 'Status 1097', 'Status 1097 Date',
       'End Date', 'Status 1098 Date', 'Planned Duration', 'Reported Duration',
       'Request Date', 'Last Update', 'EA Number', 'Facility', 'Closure Lanes',
       'Total Lanes', 'DB ID', 'StartDateTime', 'EndDateTime'],
      dtype='object')
(2007, 28)


,ID,Log #,District,Fwy-Dir,Begin County,Begin Abs PM,Begin State PM,End County,End Abs PM,End State PM,Length,Work Type,Start Date,Status 1097,Status 1097 Date,End Date,Status 1098 Date,Planned Duration,Reported Duration,Request Date,Last Update,EA Number,Facility,Closure Lanes,Total Lanes,DB ID,StartDateTime,EndDateTime
0,C128FA,4,3,SR128-E,95.0,112.493,0.000,113.0,117.083,4.590,4.6,Emergency,2020-10-01 6:01:00,Y,2020-10-01 7:03:00,2020-10-01 18:01:00,2020-10-01 17:18:00,720,616,2020-9-22 9:55:00,2020-10-01 17:18:53,0321000053,Conventional Hwy,1,2,11523466,2020-10-01 06:01:00,2020-10-01 18:01:00
1,C128FA,5,3,SR128-E,95.0,112.493,0.000,113.0,117.083,4.590,4.6,Emergency,2020-10-02 6:01:00,Y,2020-10-02 7:04:00,2020-10-02 18:01:00,2020-10-02 16:59:00,720,595,2020-9-22 9:56:00,2020-10-02 16:59:48,0321000053,Conventional Hwy,1,2,11523469,2020-10-02 06:01:00,2020-10-02 18:01:00
6,C12GA,1,3,SR12-E,95.0,63.596,0.000,67.0,64.167,0.571,0.6,Electrical,2020-10-05 20:01:00,Y,2020-10-05 22:48:00,2020-10-06 4:01:00,2020-10-06 2:48:00,480,240,2020-9-29 11:20:00,2020-10-06 02:48:33,0317000167,Conventional Hwy,1,2,11542256,2020-10-05 20:01:00,2020-10-06 04:01:00
7,C12GA,2,3,SR12-E,95.0,63.596,0.000,67.0,64.167,0.571,0.6,Electrical,2020-10-06 20:01:00,Y,2020-10-06 21:37:00,2020-10-07 4:01:00,2020-10-07 4:13:00,480,396,2020-9-29 11:20:00,2020-10-07 04:13:27,0317000167,Conventional Hwy,1,2,11542258,2020-10-06 20:01:00,2020-10-07 04:01:00
8,C12GA,3,3,SR12-E,95.0,63.596,0.000,67.0,64.167,0.571,0.6,Electrical,2020-10-07 20:01:00,Y,2020-10-07 20:19:00,2020-10-08 4:01:00,2020-10-08 3:59:00,480,460,2020-9-29 11:20:00,2020-10-08 03:59:02,0317000167,Conventional Hwy,1,2,11542259,2020-10-07 20:01:00,2020-10-08 04:01:00
9,C12GA,4,3,SR12-E,95.0,63.596,0.000,67.0,64.167,0.571,0.6,Electrical,2020-10-08 20:01:00,Y,2020-10-08 20:28:00,2020-10-09 4:01:00,2020-10-09 4:02:00,480,454,2020-9-29 11:20:00,2020-10-09 04:02:36,0317000167,Conventional Hwy,1,2,11542261,2020-10-08 20:01:00,2020-10-09 04:01:00
24,C160CA,1,3,SR160-N,67.0,19.086,6.976,67.0,19.492,7.382,0.4,Bridge,2020-10-10 6:01:00,Y,2020-10-10 7:05:00,2020-10-10 15:01:00,2020-10-10 14:30:00,540,445,2020-10-09 8:50:00,2020-10-10 14:50:32,0319000123,Conventional Hwy,1,2,11568926,2020-10-10 06:01:00,2020-10-10 15:01:00
25,C160CA,5,3,SR160-N,67.0,32.982,20.872,67.0,32.982,20.872,0.0,Bridge,2020-10-19 19:01:00,Y,2020-10-19 19:00:00,2020-10-20 6:01:00,2020-10-20 3:30:00,660,510,2020-10-12 14:08:00,2020-10-20 03:30:02,0319000123,Conventional Hwy,2,2,11577734,2020-10-19 19:01:00,2020-10-20 06:01:00
26,C160CA,6,3,SR160-N,67.0,32.982,20.872,67.0,32.982,20.872,0.0,Bridge,2020-10-20 19:01:00,Y,2020-10-20 19:05:00,2020-10-21 6:01:00,2020-10-21 2:33:00,660,449,2020-10-12 14:08:00,2020-10-21 02:33:57,0319000123,Conventional Hwy,2,2,11577739,2020-10-20 19:01:00,2020-10-21 06:01:00
27,C160CA,3,3,SR160-N,67.0,31.865,19.755,67.0,31.865,19.755,0.0,Bridge,2020-10-21 19:01:00,Y,2020-10-21 19:03:00,2020-10-22 6:01:00,2020-10-22 4:28:00,660,565,2020-10-12 14:04:00,2020-10-22 04:28:26,0319000123,Conventional Hwy,2,2,11577639,2020-10-21 19:01:00,2020-10-22 06:01:00


In [8]:

station_meta = merged[['Timestamp', 'StationID', 'FreewayNum', 'DirectionOfTravel', 'LaneType',
       'StationLength', 'Samples', '%Observed', 'County', 'TotalFlow', 'AvgOccupancy',
       'AvgSpeed', 'flowPerLane', 'Hour', 'Day', 
       'State_PM', 'Abs_PM', 'Length', 'Lanes', 'Name']]

station_meta['Fwy-Dir'] = ''
#for i in range(len(station_meta.head())):
#  station_meta['Fwy-Dir'].iloc[i] = (str(station_meta['FreewayNum'].iloc[i])) + '-' + station_meta['DirectionOfTravel'].iloc[i]
station_meta['Fwy-Dir'] = station_meta['FreewayNum'].apply(str) + '-' + station_meta['DirectionOfTravel']

station_meta['lanes_closed'] = 0
station_meta['work type'] = 0
station_meta['closed station'] = False
station_meta['DateTime'] =  pd.to_datetime(station_meta['Timestamp'])

print(station_meta.shape)
print(station_meta.columns)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

(441071, 25)
Index(['Timestamp', 'StationID', 'FreewayNum', 'DirectionOfTravel', 'LaneType',
       'StationLength', 'Samples', '%Observed', 'County', 'TotalFlow',
       'AvgOccupancy', 'AvgSpeed', 'flowPerLane', 'Hour', 'Day', 'State_PM',
       'Abs_PM', 'Length', 'Lanes', 'Name', 'Fwy-Dir', 'lanes_closed',
       'work type', 'closed station', 'DateTime'],
      dtype='object')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [11]:
#for i in station_meta.head():
#  print(i)

#for i in range(len(station_meta.head())):
#  station_meta['Fwy-Dir'].iloc[i] = (str(station_meta['FreewayNum'].iloc[i])) + '-' + station_meta['DirectionOfTravel'].iloc[i]


station_meta.sample(10)

#for coef, feat in zip(station_meta.head(), closures.head()):
#    print(coef, feat)

,Timestamp,StationID,FreewayNum,DirectionOfTravel,LaneType,StationLength,Samples,%Observed,County,TotalFlow,AvgOccupancy,AvgSpeed,flowPerLane,Hour,Day,State_PM,Abs_PM,Length,Lanes,Name,Fwy-Dir,lanes_closed,work type,closed station,DateTime
204206,10/15/2020 14:00:00,316597,65,S,ML,0.382,240,100,61,2696.0,0.2383,25.6,1348.000000,14,15,R7.613,67.331,0.382,2,Pleasant Grove Blvd,65-S,0,0,False,2020-10-15 14:00:00
336365,10/04/2020 05:00:00,318711,5,S,ML,0.375,360,0,67,1247.0,0.0414,64.0,415.666667,5,4,29.139,524.432,0.375,3,5SB at Del Paso Rd,5-S,0,0,False,2020-10-04 05:00:00
100129,10/19/2020 01:00:00,313852,50,E,ML,0.694,480,100,67,326.0,0.0042,67.5,81.500000,1,19,R7.055,12.689,0.694,4,Mayhew Rd,50-E,0,0,False,2020-10-19 01:00:00
187022,10/12/2020 14:00:00,316160,80,W,ML,0.000,480,0,67,3667.0,0.0745,62.8,916.750000,14,12,M4.38,88.285,0.000,4,North Freeway,80-W,0,0,False,2020-10-12 14:00:00
164108,10/18/2020 20:00:00,315006,5,S,ML,0.493,598,100,67,2718.0,0.0349,73.1,543.600000,20,18,26.175,521.468,0.493,5,5SB at W El Camino,5-S,0,0,False,2020-10-18 20:00:00
251264,10/23/2020 08:00:00,317008,80,E,ML,0.987,351,0,61,2846.0,0.0752,59.7,948.666667,8,23,7.118,108.618,0.987,3,W. of Sierra College,80-E,0,0,False,2020-10-23 08:00:00
99915,10/10/2020 03:00:00,313852,50,E,ML,0.694,475,100,67,357.0,0.0051,67.5,89.250000,3,10,R7.055,12.689,0.694,4,Mayhew Rd,50-E,0,0,False,2020-10-10 03:00:00
162139,10/29/2020 19:00:00,314968,5,N,ML,0.575,480,100,67,2465.0,0.0547,63.5,616.250000,19,29,20.585,515.878,0.575,4,Sutterville Rd,5-N,0,0,False,2020-10-29 19:00:00
175647,10/03/2020 15:00:00,316009,80,E,ML,0.324,360,100,67,3358.0,0.0720,65.9,1119.333333,15,3,M7.768,91.673,0.324,3,80EB at Marysville,80-E,0,0,False,2020-10-03 15:00:00
244957,10/08/2020 13:00:00,316949,50,E,ML,0.584,0,0,67,1651.0,0.1062,48.3,825.500000,13,8,19.953,25.623,0.584,2,E. of PrairieCity Rd,50-E,0,0,False,2020-10-08 13:00:00


In [10]:
closures.sample(3)

,ID,Log #,District,Fwy-Dir,Begin County,Begin Abs PM,Begin State PM,End County,End Abs PM,End State PM,Length,Work Type,Start Date,Status 1097,Status 1097 Date,End Date,Status 1098 Date,Planned Duration,Reported Duration,Request Date,Last Update,EA Number,Facility,Closure Lanes,Total Lanes,DB ID,StartDateTime,EndDateTime
903,C80OD,13,3,I80-E,61.0,119.041,17.541,61.0,119.041,17.541,0.0,Electrical,2020-10-27 21:01:00,Y,2020-10-27 18:27:00,2020-10-28 5:01:00,2020-10-28 5:12:00,480,645,2020-10-20 14:09:00,2020-10-28 05:24:17,0300020616,Off Ramp,2,2,11603300,2020-10-27 21:01:00,2020-10-28 05:01:00
1514,M50JA,2,3,US50-W,67.0,4.349,1.193,67.0,4.349,1.193,0.0,Barrier,2020-10-15 21:01:00,Y,2020-10-15 21:57:00,2020-10-16 1:01:00,2020-10-16 1:09:00,240,192,2020-10-15 14:13:00,2020-10-16 01:09:02,03-M000493,On Ramp,1,1,11588360,2020-10-15 21:01:00,2020-10-16 01:01:00
1617,M89EA,1,3,SR89-N,91.0,115.100,25.891,91.0,118.793,29.584,3.7,Roadside,2020-10-06 6:30:00,Y,2020-10-06 9:30:00,2020-10-06 15:30:00,2020-10-06 16:05:00,540,396,2020-10-05 8:24:00,2020-10-06 16:05:54,03-M000536,Conventional Hwy,1,2,11553542,2020-10-06 06:30:00,2020-10-06 15:30:00


In [14]:

#for i in range(len(station_meta.head())):
#  for j in range(len(closures)):
#    print(station_meta['Fwy-Dir'].iloc[i])
#    if (station_meta['Fwy-Dir'].iloc[i] in closures['Fwy-Dir'].iloc[j]):
      #station_meta['lanes_closed'].iloc[i] = closures['Closure Lanes'].iloc[j]
      #station_meta['work type'].iloc[i] = closures['Work Type'].iloc[j]
      #station_meta['closed station'].iloc[i] = True
#      print(i,j)


#for i in range(len(station_meta.sample(10))):
#  for j in range(len(closures.sample(10))):
#    print(closures['Begin Abs PM'].iloc[j] , station_meta['Abs_PM'].iloc[i] , closures['End Abs PM'].iloc[j])
    #if (closures['Begin Abs PM'].iloc[j] < station_meta['Abs_PM'].iloc[i] < closures['End Abs PM'].iloc[j]):

      #(closures['StartDateTime'].iloc[j] < station_meta['DateTime'].iloc[i] < closures['EndDateTime'].iloc[j])):
      #station_meta['lanes_closed'].iloc[i] = closures['Closure Lanes'].iloc[j]
#      station_meta['work type'].iloc[i] = closures['Work Type'].iloc[j]
#      station_meta['closed station'].iloc[i] = True
      #print(i,j)

print(station_meta['lanes_closed'].sum())

station_meta.sample(10)['Abs_PM']

0


384716     70.451
325951      2.900
309868      5.284
130482     95.877
325818    512.695
92442      15.102
301591     76.172
390047     85.310
411715     21.405
402582     96.560
Name: Abs_PM, dtype: float64

In [ ]:
for i in range(len(station_meta)):
  for j in range(len(closures)):
    a,b,c,d = False, False, False, False
    if (station_meta['Fwy-Dir'].iloc[i] in closures['Fwy-Dir'].iloc[j]):
      a = True
    if (closures['Begin Abs PM'].iloc[j] < station_meta['Abs_PM'].iloc[i] < closures['End Abs PM'].iloc[j]):
      b = True
    if (closures['Begin Abs PM'].iloc[j] > station_meta['Abs_PM'].iloc[i] > closures['End Abs PM'].iloc[j]):
      c = True
    if (closures['StartDateTime'].iloc[j] < station_meta['DateTime'].iloc[i] < closures['EndDateTime'].iloc[j]):
      d = True
    if (a & (b | c) & d):
      station_meta['lanes_closed'].iloc[i] = closures['Closure Lanes'].iloc[j]
      station_meta['work type'].iloc[i] = closures['Work Type'].iloc[j]
      station_meta['closed station'].iloc[i] = True
      print(i,j)

print(station_meta['lanes_closed'].sum())



for i in range(len(station_meta)):
  for j in range(len(closures)):
    if ((station_meta['Fwy-Dir'].iloc[i] in closures['Fwy-Dir'].iloc[j]) & 
      ((closures['Begin Abs PM'].iloc[j] < station_meta['Abs_PM'].iloc[i] < closures['End Abs PM'].iloc[j])
      | (closures['Begin Abs PM'].iloc[j] > station_meta['Abs_PM'].iloc[i] > closures['End Abs PM'].iloc[j])) &
      (closures['StartDateTime'].iloc[j] < station_meta['DateTime'].iloc[i] < closures['EndDateTime'].iloc[j])):
      station_meta['lanes_closed'].iloc[i] = closures['Closure Lanes'].iloc[j]
      station_meta['work type'].iloc[i] = closures['Work Type'].iloc[j]
      station_meta['closed station'].iloc[i] = True
      print(i,j)

print(station_meta['lanes_closed'].sum())